In [17]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [3]:
marg_90 = pd.read_csv("Marginacion_1990.csv")

In [7]:
variables = marg_90[['ANALF','SPRIM','OVSDE','OVSEE','OVSAE','VHAC','OVPT','PL.5000','PO2SM']]
#variables_np = variables.to_numpy()
var_cov_std = StandardScaler().fit_transform(variables)
features = var_cov_std.T
var_cov = np.cov(features)
var_cov.shape

(9, 9)

In [5]:
def sign(x):
    """
    Helper function for computing sign of real number x.
    """
    if x >=0:
        return 1
    else:
        return -1

In [55]:
def compute_cos_sin_Jacobi_rotation(Ak, idx1, idx2):
    """
    Helper function for computing entries of Jacobi rotation.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        c (float): value of cos of theta for Jacobi rotation matrix.
        s (float): value of sin of theta for Jacobi rotation matrix.
    """
    if np.abs(Ak[idx1,idx2]) >= np.finfo(float).eps:
        tau = (Ak[idx2, idx2] - Ak[idx1, idx1])/(2*Ak[idx1, idx2])
        t_star = sign(tau)/(np.abs(tau) + np.sqrt(1+tau**2))
        c = 1/np.sqrt(1+t_star**2)
        s = c*t_star
    else: #no rotation is performed
        c = 1
        s = 0
    return (c,s)

In [48]:
def off(Ak):
    """
    Frobenius norm without the main diagonal
    Args:
        Ak (numpy ndarray): Matrix for getting the Frobenius norm.
    Returns:
        s: The Frobenius norm without the sum of the main diagonal indices.
    """
    n = int(np.sqrt(np.size(var_cov)))
    s = 0
    for i in range(n):
        for j in range(n):
            if j != i:
                s1 = math.sqrt(Ak[i,j]**2)
                s = s + s1
            else:
                pass
    return s

In [39]:
var_cov[1,2]**2

0.7544138346118824

In [8]:
def compute_Jacobi_rotation(Ak, idx1, idx2):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        J (numpy ndarray): Jacobi rotation matrix.
    """
    c,s = compute_cos_sin_Jacobi_rotation(Ak, idx1, idx2)
    m,n = Ak.shape
    J = np.eye(m)
    J[idx1, idx1] = J[idx2, idx2] = c
    J[idx1, idx2] = s
    J[idx2, idx1] = -s
    return J

In [1]:
def compute_Jacobi_egvalues(A, sweeps):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        J (numpy ndarray): Jacobi rotation matrix.
    """
    Ak = A.copy()
    m, n = Ak.shape 
    for n in range(sweeps):
        for i in range(Ak.shape[0]):
            j = m -1
            for m in range(Ak.shape[0]-i-1):
                if off(Ak) > (1e-8)*np.linalg.norm(Ak) and j >= 0:
                    J = compute_Jacobi_rotation(Ak, i, j)
                    Ak = J.T@Ak@J
                    j -= 1
                    
    print(Ak)

In [10]:
def compute_Jacobi_egvectors(A, sweeps):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        Q (numpy ndarray): Matrix that contain A's eigenvectors.
    """
    m,n = A.shape
    Q = np.eye(m)
    for n in range(sweeps):
        for i in range(A.shape[0]):
            j = i + 1
            for m in range(A.shape[0]-i-1):
                if (off(A) > (1e-8)*np.linalg.norm(A)) :
                    J = compute_Jacobi_rotation(A, i, j)
                    Qk = Q@J
                    Q = Qk
                    j += 1
                    
    return Qk

In [30]:
values, vectors = np.linalg.eig(var_cov)
values

array([7.3  , 0.611, 0.431, 0.325, 0.036, 0.233, 0.09 , 0.145, 0.118])

In [3]:
A = np.ones((4,4))
A

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [57]:
def compute_Jacobi_eg2(A, max_sweeps, tol):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        Q (numpy ndarray): Matrix that contain A's eigenvectors.
    """
    Ak = A.copy()
    m,n = Ak.shape
    Q = np.eye(m)
    sweeps = 0
    for n in range(sweeps):
        for i in range(Ak.shape[0]):
            j = m -1
            for m in range(Ak.shape[0]-i-1):
                while (off(A_k) > tol*np.linalg.norm(Ak) and sweeps < max_sweep):
                    J = compute_Jacobi_rotation(Ak, i, j)
                    Q = Q@J
                    #Q = Qk
                    j -= 1
                    sweeps += 1
                    
    return Q

In [58]:
def compute_Jacobi_eg3(A, sweeps, tol):
    m,n = A.shape
    Q = np.eye(m)
    max_sweeps = 11
    for n in range(sweeps):
        for i in range(A.shape[0]):
            j = m -1
    
while off(A_k) > tol ||A_k||_F && sweeps < max_sweeps

SyntaxError: invalid syntax (<ipython-input-58-6358889274f3>, line 9)

In [67]:
vectors

array([[-0.356,  0.071, -0.241, -0.266, -0.674,  0.26 , -0.343, -0.299,
        -0.106],
       [-0.355, -0.006, -0.124,  0.265,  0.342, -0.127, -0.145, -0.153,
        -0.782],
       [-0.336, -0.368, -0.102,  0.365,  0.332,  0.311, -0.222, -0.327,
         0.495],
       [-0.317,  0.509,  0.429,  0.099,  0.1  ,  0.237, -0.395,  0.466,
         0.093],
       [-0.329,  0.293,  0.391, -0.19 ,  0.067, -0.515,  0.15 , -0.53 ,
         0.21 ],
       [-0.33 , -0.151, -0.457, -0.377,  0.179, -0.47 , -0.212,  0.421,
         0.207],
       [-0.34 ,  0.3  , -0.287, -0.221,  0.208,  0.429,  0.659,  0.046,
         0.012],
       [-0.343, -0.077, -0.022,  0.608, -0.477, -0.268,  0.357,  0.268,
         0.082],
       [-0.289, -0.63 ,  0.537, -0.344, -0.046,  0.15 ,  0.163,  0.177,
        -0.175]])

In [12]:
max_sweeps = 3
for n in range(2):
        for i in range(A.shape[0]):
            j = i + 1
            for m in range(A.shape[0]-i-1):
                while off(A_k) > tol ||A_k||_F && sweeps < max_sweep:
                    #J = compute_Jacobi_rotation(A, i, j)
                    #Q = Q@J
                    print(i,j)
                    j += 1

0 1
0 2
0 3
1 2
1 3
2 3
0 1
0 2
0 3
1 2
1 3
2 3


In [15]:
o,n = A.shape
for i in range(A.shape[0]):
            j = o -1
            for m in range(A.shape[0]-i):
                if off(A) > (1e-8)*np.linalg.norm(A) and j >= 0:
                    print(m,i)

0 0
1 0
2 0
3 0
0 1
1 1
2 1
0 2
1 2
0 3


In [45]:
def compute_Jacobi_egvectors4(A, max_sweeps):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        max_sweeps (int):  the max number of times the process is carried out.
    Returns:
        Q (numpy ndarray): Matrix that contain A's eigenvectors.
    """
    Ak = A.copy
    m,n = A.shape
    Q = np.eye(m)
    sweeps = 0
    for n in range(max_sweeps):
        for i in range(n):
            j = m -1
            for m in range(n-i-1):
                if off(Ak) > (1e-8)*np.linalg.norm(Ak) and sweeps < max_sweep:
                    J = compute_Jacobi_rotation(Ak, i, j)
                    Q = Q@J
                    #Q = Qk
                    j -= 1
                    sweeps += 1
    return Q

In [42]:
#compute_Jacobi_egvectors4(var_cov,4,10e-8)
off(var_cov)

56.204774899483766

In [21]:
#while (off(Ak) > 10e-8**np.linalg.norm(Ak) and ) 
type(var_cov)

numpy.ndarray

In [8]:
val, vec = np.linalg.eig(var_cov)
vec

array([[-0.356,  0.071, -0.241, -0.266, -0.674,  0.26 , -0.343, -0.299,
        -0.106],
       [-0.355, -0.006, -0.124,  0.265,  0.342, -0.127, -0.145, -0.153,
        -0.782],
       [-0.336, -0.368, -0.102,  0.365,  0.332,  0.311, -0.222, -0.327,
         0.495],
       [-0.317,  0.509,  0.429,  0.099,  0.1  ,  0.237, -0.395,  0.466,
         0.093],
       [-0.329,  0.293,  0.391, -0.19 ,  0.067, -0.515,  0.15 , -0.53 ,
         0.21 ],
       [-0.33 , -0.151, -0.457, -0.377,  0.179, -0.47 , -0.212,  0.421,
         0.207],
       [-0.34 ,  0.3  , -0.287, -0.221,  0.208,  0.429,  0.659,  0.046,
         0.012],
       [-0.343, -0.077, -0.022,  0.608, -0.477, -0.268,  0.357,  0.268,
         0.082],
       [-0.289, -0.63 ,  0.537, -0.344, -0.046,  0.15 ,  0.163,  0.177,
        -0.175]])

In [49]:
compute_Jacobi_egvectors4(var_cov, 4)

TypeError: 'builtin_function_or_method' object is not subscriptable

In [18]:
def compute_Jacobi_egvectors3(A, tol):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        Q (numpy ndarray): Matrix that contain A's eigenvectors.
    """
    Ak = A.copy
    m,n = Ak.shape
    Q = np.eye(m)
    sweeps = 0
    while off(Ak) > tol*np.linalg.norm(Ak) and sweeps < 11:
        sweeps += 1
        for i in range(n):
            j = i + 1
            for m in range(A.shape[0]-i-1):
                J = compute_Jacobi_rotation(Ak, i, j)
                Q = Q@J
                j += 1
    return Q